# Bach Chorale Generator

In [1]:
import os
import sys
import pathlib
import urllib
import shutil
import re
import zipfile

import numpy as np
import torch
import music21

Create dataset of Chorale phrase tensors. Each item in the dataset is a vector
comprised of several concatenated one-hot vectors, where each one defines a
feature in the phrase

In [2]:
from ChoralePhraseTensor import PhraseTensor
from abstract_chorale_objects import ChoraleVector

In [3]:
CREATE_PHRASE_TENSOR_DATASET = False

if CREATE_PHRASE_TENSOR_DATASET:

    # Get an instance of the Chorale iterator, to iterate over all chorales in corpus
    chorales_iterator = music21.corpus.chorales.Iterator()

    # Make a ChoraleVector object for each of the chorales
    chorale_vectors = {}
    for chorale in chorales_iterator:
        new_chorale_vector = ChoraleVector(chorale)
        chorale_vectors[new_chorale_vector.chorale_name] = new_chorale_vector


    # Remove problematic outlier chorale
    # TODO: Rethink some of these chorales and see whether some change to the data processing could be
    # incorporated s.t. some of these could be returned to our data set
    # problematic_chorales = [
    #     'bwv8.6.mxl',
    #     'bwv41.6.mxl',
    #     'bwv29.8.mxl',
    #     'bwv407.mxl'
    # ]
    #
    # for chorale in problematic_chorales:
    #     if chorale in chorale_vectors.keys():
    #         del chorale_vectors[chorale]


    # Create a tensor dataset from all phrase tensors extracted from chorales
    phrase_tensors = []
    for chorale_name in chorale_vectors.keys():
        for phrase in chorale_vectors[chorale_name].phrase_object_vector:
            new_phrase_tensor_object = PhraseTensor(phrase)
            if not new_phrase_tensor_object.valid:
                continue
            phrase_tensors.append(new_phrase_tensor_object().unsqueeze(0))

    phrase_tensor_dataset = torch.utils.data.TensorDataset(torch.concat([*phrase_tensors], dim=0))

    torch.save(phrase_tensor_dataset, 'phrase_tensor_dataset.pt')

In [4]:
phrase_tensor_dataset = torch.load('phrase_tensor_dataset.pt')

In [7]:
# Now we train the generator to produce such phrases

# First, set hyperparameters for learning:
hp = dict(
    batch_size=10, z_dim=10, x_sigma2=1e-3, learn_rate=1e-4, betas=(0.5, 0.999),
)

import torch.optim as optim
from torch.utils.data import random_split
from torch.utils.data import DataLoader
from torch.nn import DataParallel
from trainer import *
import trainer
from phrase_encoder_decoder import VAE, PhraseEncoder, \
    PhraseDecoder, vae_loss, PhraseDecoderWithSoftmax

torch.manual_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

# Hyperparams
batch_size = hp['batch_size']
z_dim = hp['z_dim']
x_sigma2 = hp['x_sigma2']
learn_rate = hp['learn_rate']
betas = hp['betas']

# Data
train_length = int(len(phrase_tensor_dataset)*0.9)
split_lengths = [train_length, len(phrase_tensor_dataset) - train_length]
ds_train, ds_test = random_split(phrase_tensor_dataset, split_lengths)
dl_train = DataLoader(ds_train, batch_size, shuffle=True)
dl_test  = DataLoader(ds_test,  batch_size, shuffle=True)
im_size = ds_train[0][0].shape[0]

# Model
encoder = PhraseEncoder(im_size, z_dim)
decoder = PhraseDecoderWithSoftmax(z_dim, im_size)
vae = VAE(encoder, decoder, im_size, z_dim)
vae_dp = DataParallel(vae).to(device)

# Optimizer
optimizer = optim.Adam(vae.parameters(), lr=learn_rate, betas=betas)

# Loss
def loss_fn(x, xr, z_mu, z_log_sigma2):
    return vae_loss(x, xr, z_mu, z_log_sigma2, x_sigma2)

# Trainer
trainer = VAETrainer(vae_dp, loss_fn, optimizer, device)
checkpoint_file = 'checkpoints/vae'
checkpoint_file_final = f'{checkpoint_file}_final'
if os.path.isfile(f'{checkpoint_file}.pt'):
    os.remove(f'{checkpoint_file}.pt')

# Show model and hypers
print(vae)
print(hp)

Using device: cpu
VAE(
  (features_encoder): PhraseEncoder(
    (encoder): Sequential(
      (0): Linear(in_features=104, out_features=57, bias=True)
      (1): ReLU()
      (2): Linear(in_features=57, out_features=33, bias=True)
      (3): ReLU()
      (4): Linear(in_features=33, out_features=10, bias=True)
    )
  )
  (features_decoder): PhraseDecoderWithSoftmax(
    (decoder): PhraseDecoder(
      (decoder): Sequential(
        (0): Linear(in_features=10, out_features=33, bias=True)
        (1): ReLU()
        (2): Linear(in_features=33, out_features=57, bias=True)
        (3): ReLU()
        (4): Linear(in_features=57, out_features=104, bias=True)
      )
    )
    (softmax): Softmax(dim=1)
  )
  (encoder_mean): Linear(in_features=10, out_features=10, bias=True)
  (log_sigma2): Linear(in_features=10, out_features=10, bias=True)
)
{'batch_size': 10, 'z_dim': 10, 'x_sigma2': 0.001, 'learn_rate': 0.0001, 'betas': (0.5, 0.999)}


In [8]:
def post_epoch_fn(epoch, train_result, test_result, verbose):
    # Plot some samples if this is a verbose epoch
    if verbose:
        samples = vae.sample(n=1)


if os.path.isfile(f'{checkpoint_file_final}.pt'):
    print(f'*** Loading final checkpoint file {checkpoint_file_final} instead of training')
    checkpoint_file = checkpoint_file_final
else:
    res = trainer.fit(dl_train, dl_test,
                      num_epochs=200, early_stopping=20, print_every=10,
                      checkpoints=checkpoint_file,
                      post_epoch_fn=post_epoch_fn)

# Plot images from best model
saved_state = torch.load(f'{checkpoint_file}.pt', map_location=device)
vae_dp.load_state_dict(saved_state['model_state'])


--- EPOCH 1/200 ---


train_batch:   0%|          | 0/189 [00:00<?, ?it/s]

test_batch:   0%|          | 0/21 [00:00<?, ?it/s]


*** Saved checkpoint checkpoints/vae.pt


train_batch:   0%|          | 0/189 [00:00<?, ?it/s]

...

--- EPOCH 200/200 ---


train_batch:   0%|          | 0/189 [00:00<?, ?it/s]

test_batch:   0%|          | 0/21 [00:00<?, ?it/s]


*** Saved checkpoint checkpoints/vae.pt


<All keys matched successfully>

In [9]:
from ChoralePhraseTensor import GeneratedPhraseTensor

for _ in range(10):
    generated_phrase = GeneratedPhraseTensor(vae_dp.module.sample(1)[0])

    generated_phrase.display_phrase_information_in_a_given_key()

***** Generated phrase information in the key of C MAJOR *****
Length in quarters = 8
Opening tonality = SUBDOMINANT
Closing tonality = DOMINANT
Opening pickup harmony = C
Opening pickup harmony soprano = C
Opening pickup harmony inversion = 0
Opening downbeat harmony = F
Opening downbeat harmony soprano = C
Opening downbeat harmony inversion = 0
Pre-fermata harmony = B
Pre-fermata harmony soprano = D
Pre-fermata harmony inversion = 0
Fermata harmony = G
Fermata harmony soprano = G
Fermata harmony inversion = 0
***** Generated phrase information in the key of A MINOR *****
Length in quarters = 8
Opening tonality = TONIC
Closing tonality = TONIC
Opening pickup harmony = A
Opening pickup harmony soprano = A
Opening pickup harmony inversion = 0
Opening downbeat harmony = A
Opening downbeat harmony soprano = E
Opening downbeat harmony inversion = 0
Pre-fermata harmony = G
Pre-fermata harmony soprano = D
Pre-fermata harmony inversion = 0
Fermata harmony = C
Fermata harmony soprano = C
Ferma